In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, kstest
import re
import geopandas
from shapely.geometry import Polygon, Point
import random
%matplotlib inline

county_data_filepath = "data/franklin_data.csv"
geodata_filepath = "data/tl_2022_39_tract.zip"
county_code = "049"

# Function to generate a random point within a polygon
def get_random_point(polygon):
    min_x, min_y, max_x, max_y = polygon.bounds

    while True:
        # Generate a random point
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        
        # Check if the point is inside the polygon
        if polygon.contains(random_point):
            return random_point

#Dictionary to describe homedata Variables
households_variables_dict = {
    "B19001_001E": "total households in tract",
    "B19001_002E": "total households under 10k",
    "B19001_003E": "total households 10k to 15k",
    "B19001_004E": "total households 15k to 20k",
    "B19001_005E": "total households 20k to 25k",
    "B19001_006E": "total households 25k to 30k",
    "B19001_007E": "total households 30k to 35k",
    "B19001_008E": "total households 35k to 40k",
    "B19001_009E": "total households 40k to 45k",
    "B19001_010E": "total households 45k to 50k",
    "B19001_011E": "total households 50k to 60k",
    "B19001_012E": "total households 60k to 75k",
    "B19001_013E": "total households 75k to 100k",
    "B19001_014E": "total households 100k to 125k",
    "B19001_015E": "total households 125k to 150k",
    "B19001_016E": "total households 150k to 200k",
    "B19001_017E": "total households 200k+"
}
households_variables_list = (
    "B19001_001E",
    "B19001_002E",
    "B19001_003E",
    "B19001_004E",
    "B19001_005E",
    "B19001_006E",
    "B19001_007E",
    "B19001_008E",
    "B19001_009E",
    "B19001_010E",
    "B19001_011E",
    "B19001_012E",
    "B19001_013E",
    "B19001_014E",
    "B19001_015E",
    "B19001_016E",
    "B19001_017E"
)

#Read csvs into pandas dataframes
county_data = pd.read_csv(county_data_filepath)
geodata = geopandas.read_file(geodata_filepath)

#Merge geographical dataframe (containing shapely ploygons) with census data
county_geodata = geodata[geodata['COUNTYFP'] == county_code]
county_geodata.to_crs(epsg=3857)
county_geodata = county_geodata.rename(columns={"TRACTCE":"tract"})
county_geodata["tract"] = county_geodata["tract"].astype(int)
county_data["tract"] = county_data["tract"].astype(int)
data = pd.merge(county_geodata, county_data, on = "tract", how="inner")
data.rename(columns=households_variables_dict, inplace = True)
households = pd.DataFrame(columns = ["id","income","location"])
total_count = 0
for index,row in data.iterrows():
    
    proportional_weights = list()
    bucket_midpoints = np.arange(2500, 197501, 5000)
    bucket_widths_proportions = np.array([2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 5, 5, 5, 10])
    weights = np.array(row["total households under 10k":"total households 150k to 200k"])
    num_buckets = 15
    for index in range(num_buckets):
        for reps in range(bucket_widths_proportions[index]):
            proportional_weights.append(weights[index]/bucket_widths_proportions[index])

    # Calculate the mean and standard deviation
    if sum(weights) == 0:
        continue
    mean = np.average(bucket_midpoints, weights=proportional_weights)
    variance = np.average((bucket_midpoints - mean) ** 2, weights=proportional_weights)
    std_dev = np.sqrt(variance)
    # Generate a normal distribution
    # Define the range for the normal distribution
    x = np.linspace(2500, 197500, 40)
    # Calculate the normal distribution
    y = norm.pdf(x, mean, std_dev) * sum(proportional_weights) * 5000
    #y_round = np.round(y)
    y_int = list(y.astype(int))
    agents_count = 0
    for income_index in range(40):
        num_agents = y_int[income_index]/10
        for agent_number in range(int(num_agents)):
            households.loc[agents_count+total_count] = {"id" : total_count+agents_count, "income" : x[income_index], "location": get_random_point(row["geometry"])}
            agents_count+=1
    total_count+=agents_count
    
households.to_csv('data/household_agents.csv', index=False)
print("done")
print(households)